In [7]:
import gym
from gym_marioai import levels

all_actions = (0,1,2,3,4,5,6,7,8,9,10,11,12)

env = gym.make('Marioai-v0', render=True,
               level_path=levels.easy_level,
               compact_observation=False, #this must stay false for proper saving in dataset
               enabled_actions=all_actions,
               rf_width=20, rf_height=10)
while True:
    done = False
    total_reward = 0
    
    #initialize data arrays with initial states for each episode
    observations = [env.reset(seed=0)]
    actions = [12] #nothing
    rewards = [0]
    terminals = [done]

    while not done:
        action = 3
        next_state, reward, done, info = env.step(action)
        
        observations.append(next_state)
        actions.append(action)
        rewards.append(reward)
        terminals.append(done)
     
        total_reward += reward

ConnectionResetError: [WinError 10054] Eine vorhandene Verbindung wurde vom Remotehost geschlossen